In [1]:
import datetime
import os
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.ensemble import AdaBoostClassifier
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score, precision_recall_curve, roc_auc_score, roc_curve)
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeClassifier

import shap

%config InlineBackend.figure_format = 'retina'

In [2]:
from notebook import notebookapp
import urllib
import json
import ipykernel
from shutil import copy2

def notebook_path():
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
    return None


def copy_current_nb(new_name):
    nb = notebook_path()
    if nb:
        new_path = os.path.join(os.path.dirname(nb), new_name+'.ipynb')
        copy2(nb, new_path)
    else:
        print("Current notebook path cannot be determined.")

In [3]:
df = pd.read_csv('Data/cover_type_engineered.csv')

In [4]:
df = df.loc[:, [col for col in df if not col.startswith('Cover_Type_')]]
X = df.drop(columns=['Cover_Type', 'Aspect_Sector'])
y = df['Cover_Type'] - 1

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M")

warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.api.types")

# Assuming X and y are defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

# Define the estimator for AdaBoost
base_estimator = DecisionTreeClassifier(max_depth=2)  # A weak learner with max_depth=2
estimator = AdaBoostClassifier(estimator=base_estimator, algorithm='SAMME')

# Define hyperparameters for tuning AdaBoost
hyperparameters = {
    "n_estimators": stats.randint(50, 500),  # The number of weak learners to train
    "learning_rate": stats.uniform(0.01, 1.0),  # The contribution of each classifier
}


random_search = RandomizedSearchCV(estimator,
                                   param_distributions=hyperparameters,
                                   scoring='accuracy',
                                   return_train_score=True,
                                   n_iter=500,
                                   cv=5,
                                   verbose=10,
                                   n_jobs=-1)

# Fit the RandomizedSearchCV
try:
    random_search.fit(X_train, y_train)  # Assuming X_train and y_train are defined
    print("Best parameters found:", random_search.best_params_)
    print("Best score found:", random_search.best_score_)

    
    # Save results
    results_path = f"./tuning_results/tuning_ada/{timestamp}"
    if not os.path.exists(results_path):
        os.makedirs(results_path)
        
    # Saving cross-validation results
    cv_results = pd.DataFrame(random_search.cv_results_)
    cv_results_file = f"{timestamp}_results.csv"
    cv_results.to_csv(os.path.join(results_path, cv_results_file), index=False)
    
    # Save .ipynb
    copy_current_nb(os.path.join(results_path, 'Evaluation_Notebook'))
    
    # Save Model
    file_name = f"ada_{timestamp}.pkl"
    pickle.dump(random_search, open(os.path.join(results_path, file_name), "wb"))
    
    # random_search = pickle.load(open(file_name, "rb"))
    

except Exception as e:
    print(f"An error occurred during model optimization: {e}")


Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV 5/5; 1/500] START learning_rate=0.7998589070408956, n_estimators=307........
[CV 5/5; 1/500] END learning_rate=0.7998589070408956, n_estimators=307;, score=(train=0.566, test=0.546) total time=  25.7s
[CV 1/5; 3/500] START learning_rate=0.6148047339746007, n_estimators=415........
[CV 1/5; 3/500] END learning_rate=0.6148047339746007, n_estimators=415;, score=(train=0.565, test=0.555) total time=  19.8s
[CV 5/5; 4/500] START learning_rate=0.5880566075667341, n_estimators=205........
[CV 5/5; 4/500] END learning_rate=0.5880566075667341, n_estimators=205;, score=(train=0.608, test=0.614) total time=   9.8s
[CV 3/5; 6/500] START learning_rate=0.3818853188581415, n_estimators=437........
[CV 3/5; 6/500] END learning_rate=0.3818853188581415, n_estimators=437;, score=(train=0.586, test=0.579) total time=  20.4s
[CV 1/5; 8/500] START learning_rate=0.576558769938067, n_estimators=385.........
[CV 1/5; 8/500] END learning_rate=0

[CV 2/5; 2/500] START learning_rate=0.8337421536399743, n_estimators=323........
[CV 2/5; 2/500] END learning_rate=0.8337421536399743, n_estimators=323;, score=(train=0.582, test=0.537) total time=  27.0s
[CV 4/5; 3/500] START learning_rate=0.6148047339746007, n_estimators=415........
[CV 4/5; 3/500] END learning_rate=0.6148047339746007, n_estimators=415;, score=(train=0.567, test=0.533) total time=  19.3s
[CV 3/5; 5/500] START learning_rate=0.47504076458216515, n_estimators=493.......
[CV 3/5; 5/500] END learning_rate=0.47504076458216515, n_estimators=493;, score=(train=0.557, test=0.548) total time=  23.3s
[CV 2/5; 7/500] START learning_rate=0.7175328613600828, n_estimators=392........
[CV 2/5; 7/500] END learning_rate=0.7175328613600828, n_estimators=392;, score=(train=0.560, test=0.541) total time=  18.6s
[CV 3/5; 8/500] START learning_rate=0.576558769938067, n_estimators=385.........
[CV 3/5; 8/500] END learning_rate=0.576558769938067, n_estimators=385;, score=(train=0.593, test=0

[CV 4/5; 1/500] START learning_rate=0.7998589070408956, n_estimators=307........
[CV 4/5; 1/500] END learning_rate=0.7998589070408956, n_estimators=307;, score=(train=0.590, test=0.563) total time=  25.9s
[CV 3/5; 3/500] START learning_rate=0.6148047339746007, n_estimators=415........
[CV 3/5; 3/500] END learning_rate=0.6148047339746007, n_estimators=415;, score=(train=0.566, test=0.558) total time=  19.9s
[CV 2/5; 5/500] START learning_rate=0.47504076458216515, n_estimators=493.......
[CV 2/5; 5/500] END learning_rate=0.47504076458216515, n_estimators=493;, score=(train=0.588, test=0.562) total time=  23.1s
[CV 4/5; 6/500] START learning_rate=0.3818853188581415, n_estimators=437........
[CV 4/5; 6/500] END learning_rate=0.3818853188581415, n_estimators=437;, score=(train=0.616, test=0.611) total time=  20.7s
[CV 1/5; 9/500] START learning_rate=0.33730815455099683, n_estimators=53........
[CV 1/5; 9/500] END learning_rate=0.33730815455099683, n_estimators=53;, score=(train=0.649, test=

[CV 2/5; 1/500] START learning_rate=0.7998589070408956, n_estimators=307........
[CV 2/5; 1/500] END learning_rate=0.7998589070408956, n_estimators=307;, score=(train=0.615, test=0.591) total time=  25.7s
[CV 2/5; 3/500] START learning_rate=0.6148047339746007, n_estimators=415........
[CV 2/5; 3/500] END learning_rate=0.6148047339746007, n_estimators=415;, score=(train=0.584, test=0.565) total time=  20.1s
[CV 1/5; 5/500] START learning_rate=0.47504076458216515, n_estimators=493.......
[CV 1/5; 5/500] END learning_rate=0.47504076458216515, n_estimators=493;, score=(train=0.582, test=0.575) total time=  23.2s
[CV 5/5; 6/500] START learning_rate=0.3818853188581415, n_estimators=437........
[CV 5/5; 6/500] END learning_rate=0.3818853188581415, n_estimators=437;, score=(train=0.608, test=0.600) total time=  20.8s
[CV 2/5; 9/500] START learning_rate=0.33730815455099683, n_estimators=53........
[CV 2/5; 9/500] END learning_rate=0.33730815455099683, n_estimators=53;, score=(train=0.667, test=

[CV 3/5; 1/500] START learning_rate=0.7998589070408956, n_estimators=307........
[CV 3/5; 1/500] END learning_rate=0.7998589070408956, n_estimators=307;, score=(train=0.604, test=0.596) total time=  25.7s
[CV 5/5; 2/500] START learning_rate=0.8337421536399743, n_estimators=323........
[CV 5/5; 2/500] END learning_rate=0.8337421536399743, n_estimators=323;, score=(train=0.547, test=0.539) total time=  15.6s
[CV 4/5; 4/500] START learning_rate=0.5880566075667341, n_estimators=205........
[CV 4/5; 4/500] END learning_rate=0.5880566075667341, n_estimators=205;, score=(train=0.619, test=0.606) total time=   9.6s
[CV 2/5; 6/500] START learning_rate=0.3818853188581415, n_estimators=437........
[CV 2/5; 6/500] END learning_rate=0.3818853188581415, n_estimators=437;, score=(train=0.620, test=0.598) total time=  20.4s
[CV 5/5; 7/500] START learning_rate=0.7175328613600828, n_estimators=392........
[CV 5/5; 7/500] END learning_rate=0.7175328613600828, n_estimators=392;, score=(train=0.564, test=0

...

[CV 4/5; 255/500] END learning_rate=0.9447480770258354, n_estimators=477;, score=(train=0.557, test=0.532) total time=  23.5s
[CV 2/5; 258/500] START learning_rate=0.5432459746663484, n_estimators=198......
[CV 2/5; 258/500] END learning_rate=0.5432459746663484, n_estimators=198;, score=(train=0.640, test=0.626) total time=   9.8s
[CV 3/5; 259/500] START learning_rate=0.8176925043244125, n_estimators=218......
[CV 3/5; 259/500] END learning_rate=0.8176925043244125, n_estimators=218;, score=(train=0.586, test=0.596) total time=  10.9s
[CV 1/5; 261/500] START learning_rate=0.2914804703297179, n_estimators=489......
[CV 1/5; 261/500] END learning_rate=0.2914804703297179, n_estimators=489;, score=(train=0.629, test=0.623) total time=  24.5s
[CV 2/5; 263/500] START learning_rate=0.9249694492683785, n_estimators=319......
[CV 2/5; 263/500] END learning_rate=0.9249694492683785, n_estimators=319;, score=(train=0.587, test=0.581) total time=  16.1s
[CV 1/5; 265/500] START learning_rate=0.557382

[CV 2/5; 245/500] END learning_rate=0.5006028897071015, n_estimators=291;, score=(train=0.625, test=0.596) total time=  14.3s
[CV 1/5; 246/500] START learning_rate=0.026135551208090517, n_estimators=375....
[CV 1/5; 246/500] END learning_rate=0.026135551208090517, n_estimators=375;, score=(train=0.664, test=0.658) total time=  18.7s
[CV 5/5; 247/500] START learning_rate=0.11127647872943257, n_estimators=282.....
[CV 5/5; 247/500] END learning_rate=0.11127647872943257, n_estimators=282;, score=(train=0.683, test=0.669) total time=  14.0s
[CV 3/5; 250/500] START learning_rate=0.3649417775343078, n_estimators=397......
[CV 3/5; 250/500] END learning_rate=0.3649417775343078, n_estimators=397;, score=(train=0.624, test=0.622) total time=  19.9s
[CV 5/5; 253/500] START learning_rate=0.2546542453421944, n_estimators=351......
[CV 5/5; 253/500] END learning_rate=0.2546542453421944, n_estimators=351;, score=(train=0.651, test=0.639) total time=  17.6s
[CV 1/5; 256/500] START learning_rate=0.375

[CV 3/5; 262/500] END learning_rate=0.2234951716665432, n_estimators=282;, score=(train=0.682, test=0.681) total time=  14.2s
[CV 1/5; 263/500] START learning_rate=0.9249694492683785, n_estimators=319......
[CV 1/5; 263/500] END learning_rate=0.9249694492683785, n_estimators=319;, score=(train=0.554, test=0.538) total time=  16.0s
[CV 4/5; 264/500] START learning_rate=0.3551470539913648, n_estimators=221......
[CV 4/5; 264/500] END learning_rate=0.3551470539913648, n_estimators=221;, score=(train=0.666, test=0.660) total time=  11.1s
[CV 2/5; 266/500] START learning_rate=0.3459743745703683, n_estimators=150......
[CV 2/5; 266/500] END learning_rate=0.3459743745703683, n_estimators=150;, score=(train=0.673, test=0.665) total time=   7.6s
[CV 2/5; 267/500] START learning_rate=0.5788335551753188, n_estimators=267......
[CV 2/5; 267/500] END learning_rate=0.5788335551753188, n_estimators=267;, score=(train=0.638, test=0.611) total time=  15.0s
[CV 5/5; 268/500] START learning_rate=0.960492

[CV 3/5; 254/500] END learning_rate=0.13341830823593304, n_estimators=172;, score=(train=0.668, test=0.659) total time=   8.6s
[CV 1/5; 255/500] START learning_rate=0.9447480770258354, n_estimators=477......
[CV 1/5; 255/500] END learning_rate=0.9447480770258354, n_estimators=477;, score=(train=0.573, test=0.536) total time=  23.6s
[CV 4/5; 257/500] START learning_rate=0.7493957142554047, n_estimators=450......
[CV 4/5; 257/500] END learning_rate=0.7493957142554047, n_estimators=450;, score=(train=0.564, test=0.529) total time=  22.5s
[CV 1/5; 260/500] START learning_rate=0.04859266852383714, n_estimators=154.....
[CV 1/5; 260/500] END learning_rate=0.04859266852383714, n_estimators=154;, score=(train=0.640, test=0.644) total time=   7.9s
[CV 3/5; 261/500] START learning_rate=0.2914804703297179, n_estimators=489......
[CV 3/5; 261/500] END learning_rate=0.2914804703297179, n_estimators=489;, score=(train=0.610, test=0.608) total time=  24.7s
[CV 4/5; 263/500] START learning_rate=0.9249

[CV 3/5; 269/500] END learning_rate=0.8704830743588918, n_estimators=350;, score=(train=0.579, test=0.569) total time=  22.8s
[CV 1/5; 271/500] START learning_rate=0.0925123365422754, n_estimators=351......
[CV 1/5; 271/500] END learning_rate=0.0925123365422754, n_estimators=351;, score=(train=0.679, test=0.678) total time=  20.5s
[CV 4/5; 272/500] START learning_rate=0.7556538965331404, n_estimators=352......
[CV 4/5; 272/500] END learning_rate=0.7556538965331404, n_estimators=352;, score=(train=0.566, test=0.542) total time=  19.3s
[CV 4/5; 274/500] START learning_rate=0.7387221336611031, n_estimators=257......
[CV 4/5; 274/500] END learning_rate=0.7387221336611031, n_estimators=257;, score=(train=0.600, test=0.580) total time=  13.3s
[CV 4/5; 276/500] START learning_rate=0.5880612598569922, n_estimators=313......
[CV 4/5; 276/500] END learning_rate=0.5880612598569922, n_estimators=313;, score=(train=0.588, test=0.556) total time=  16.3s
[CV 2/5; 278/500] START learning_rate=0.904880

In [ ]:
cv_results.head()

In [ ]:
results_path = f"./tuning_results/tuning_ada/{timestamp}/Assets"
if not os.path.exists(results_path):
    os.makedirs(results_path)

# CV Evaluation

In [ ]:
cv_results.columns

In [ ]:
cv_results.sort_values(by='rank_test_score', ascending=True).head(5)

In [ ]:
sorted_cv = cv_results.sort_values(by='rank_test_score', ascending=True)

# Train vs Test Comparison

In [ ]:
plt.figure(figsize=(16, 6))    

plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_train_score'], label="Train Score")
plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_test_score'], label="Test Score")

plt.grid()
plt.xlabel('Sorted Test Rank')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy by Final Rank (top 15)')
plt.legend(loc='best')

filename = "test_train_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Test Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_test_score', 'split1_test_score', 'split2_test_score',
   'split3_test_score', 'split4_test_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "test_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Train Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_train_score', 'split1_train_score', 'split2_train_score',
   'split3_train_score', 'split4_train_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "train_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
max_params = cv_results.loc[cv_results['rank_test_score'] == 1]
best_params = max_params.params.values[0]

In [ ]:
print(f"Mean Train set, Accuracy = {max_params['mean_train_score'].values[0]:.2f}")
print(f"Mean Test  set, Accuracy = {max_params['mean_test_score'].values[0]:.2f}")

In [ ]:
random_search = pickle.load(open(os.path.join(results_path, file_name), "rb"))
model = random_search.best_estimator_

#model = ADAClassifier(**best_params)
#model.fit(X_train, y_train)

y_train_prediction = model.predict(X_train)
y_test_prediction = model.predict(X_test)

In [ ]:
print(f"Train set, Accuracy = {accuracy_score(y_train, y_train_prediction):.2f}")
print(f"Test set, Accuracy = {accuracy_score(y_test, y_test_prediction):.2f}")

In [ ]:
ind = np.argpartition(model.feature_importances_, -20)[-20:]

features = X.columns[ind]
importance = model.feature_importances_[ind]

fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(ind)), importance, align='center')
plt.yticks(range(len(ind)), features)
plt.title('Feature Importance ADA')
plt.grid()

filename = "feature_importance.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()

In [ ]:
# TEST
for col in ['param_n_estimators', 'param_learning_rate']:
    
    plt.figure(figsize=(16, 6))    

    m, b = np.polyfit(list(sorted_cv['rank_test_score'].values), list(sorted_cv[col].values), 1)
    plt.plot(sorted_cv['rank_test_score'], m * sorted_cv['rank_test_score'] + b, c='r', label="Regression Line")
    plt.plot(sorted_cv['rank_test_score'], sorted_cv[col], label=f"{col} Values")
    
    plt.grid()
    plt.xlabel('Hyperparameter Values')
    plt.ylabel('Parameter Value')
    plt.title(col)
    plt.legend(loc='best')

    
    filename = f"{col}_by_rank.png"
    plt.savefig(os.path.join(results_path, filename))
                  
    plt.show()


# Hyperparameter Evaluation

In [ ]:

def plot_parameters(x_values, title):
    
    fig, ax1 = plt.subplots(figsize=(16, 6))
    ax2 = ax1.twinx()

    ax1.scatter(x_values, cv_results['mean_test_score'], label='mean_test_score', c='b')
    ax2.scatter(x_values, cv_results['std_test_score'], label='std_test_score', c='r')

    m, b = np.polyfit(list(x_values.values), list(cv_results['mean_test_score'].values), 1)
    ax1.plot(x_values, m * x_values + b, c='b')

    m, b = np.polyfit(list(x_values.values), list(cv_results['std_test_score'].values), 1)
    ax2.plot(x_values, m * x_values + b, c='r')
    
    ax1.set_title(title)
    ax1.set_xlabel('Parameter Value')
    ax1.set_ylabel('Mean Test Score')
    ax2.set_ylabel('Standard Deviation of Test Score')
    ax1.grid(True)
    
    
    # Combine the legends from both axes
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')

    filename = f"{title}_test_score.png"
    plt.savefig(os.path.join(results_path, filename))
            
    plt.show()



In [ ]:

for param in ['param_n_estimators', 'param_learning_rate']:
    x_values = cv_results[param]
    plot_parameters(x_values, param)

# Plotting Evaluation Metrics (Precision, Recall, F1-Score, AUC-ROC):


In [ ]:

# For multiclass classification, you need to binarize the labels
y_true_bin = label_binarize(y_test, classes=np.unique(y_test))
y_score_bin = label_binarize(y_test_prediction, classes=np.unique(y_test_prediction))

auc_roc = roc_auc_score(y_true_bin, y_score_bin, average='macro')

# Plot Precision-Recall curve for each class
precision = dict()
recall = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(recall[i], precision[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('Recall')
plt.ylabel('True Positive Rate / Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')

filename = "precision_recall.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Plot AUC-ROC curve for each class
fpr = dict()
tpr = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(fpr[i], tpr[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate / Precision')
plt.title('ROC Curve')
plt.legend(loc='best')
            
filename = "roc_curve.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Partial Dependence

In [ ]:
# potentially iterate over features (and relation ie 0 to 1)

In [ ]:

features, feature_names = [(0,)], [f"Features #{i}" for i in range(X.shape[1])]
deciles = {0: np.linspace(0, 1, num=5)}

pd_results = partial_dependence(
    model, X, features=1, kind="average", grid_resolution=5)

display = PartialDependenceDisplay(
    [pd_results], features=features, feature_names=feature_names,
    target_idx=0, deciles=deciles
)
display.plot(pdp_lim={1: (-1.38, 0.66)})

plt.grid()
plt.xlabel('Feature Value')
plt.ylabel('Partial Dependence') 
plt.title('Partial Dependence')

filename = "partial_dependence.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Confusion Matrix

In [ ]:

class_names = np.unique(y)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [
    ("Confusion matrix without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        X_test,
        y_test,
        display_labels=class_names + 1,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    png_name = title.lower().replace(" ", "_")
    filename = f"{png_name}.png"
    plt.savefig(os.path.join(results_path, filename))

plt.show()

# Shap Values

In [ ]:
explainer = shap.TreeExplainer(model)
explanation = explainer.shap_values(X_test, check_additivity=False)


In [ ]:
shap.summary_plot(explanation, X_test, plot_type="bar", show=False)

filename = f"shap_summary.png"
plt.savefig(os.path.join(results_path, filename))
plt.close()  

SHAP values show how each feature affects each final prediction, the significance of each feature compared to others, and the model's reliance on the interaction between features.
